<a href="https://colab.research.google.com/github/OrleiSan/Pipeline_SQL/blob/main/Pipeline_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial de apoio

In [3]:
# Tutorial de apoio para questão 7
from IPython.display import HTML

video_id = "vUBkZLpLjq4"  # ID do vídeo
video_url = f"https://www.youtube.com/embed/{video_id}"

HTML(f"""
<iframe width="560" height="315" src="{video_url}" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
""")

# Conexão

In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.4 MB/s eta 0:00:00


In [2]:
!curl ipecho.net/plain

35.194.82.160

#Buscando o arquivo via API para gerar o banco de dados a partir um arquivo CSV.

In [5]:
import requests
import pandas as pd
import json

In [ ]:
# Termo de busca
termo_de_busca = "webcam"

# URL da API do Mercado Livre
url = f"https://api.mercadolibre.com/sites/MLB/search?q={termo_de_busca}"

# Fazer a requisição GET
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Carregar os dados em formato JSON
    dados = response.json()

    # Extrair os dados relevantes
    produtos = []
    for item in dados.get('results', []):
        produto = {
            "Título": item.get('title'),
            "Preço (R$)": item.get('price'),
            "Link": item.get('permalink'),
            "Condição": item.get('condition'),
            "Categoria": item.get('category_id')
        }
        produtos.append(produto)

    # Criar o DataFrame
    df = pd.DataFrame(produtos)

    # Exibir o DataFrame
    display(df)
else:
    print(f"Erro na requisição: {response.status_code}")

# Tratando com o pandas

In [7]:
df.head(1)

,Título,Preço (R$),Link,Condição,Categoria
0,"Webcam Home Ofice ! Microfone ! Teans, Zoom, M...",54.46,https://produto.mercadolivre.com.br/MLB-186361...,new,MLB73364


In [8]:
df['Condição'] = df['Condição'].replace('new', 'novo')

In [9]:
df.head(1)

,Título,Preço (R$),Link,Condição,Categoria
0,"Webcam Home Ofice ! Microfone ! Teans, Zoom, M...",54.46,https://produto.mercadolivre.com.br/MLB-186361...,novo,MLB73364


In [16]:
df.rename(columns={'titulo': 'produto',
                   'Preço (R$)': 'preco',
                   'Link': 'link',
                   'Condição': 'condicao',
                   'Categoria': 'categoria'}, inplace=True)
df.head(1)

,produto,preco,link,condicao,categoria
0,"Webcam Home Ofice ! Microfone ! Teans, Zoom, M...",54.46,https://produto.mercadolivre.com.br/MLB-186361...,novo,MLB73364


# Conexão com o banco SQL

In [14]:
import mysql.connector
from mysql.connector import Error

In [ ]:
'''
7 - Crie um database SQL chamado mercado em sua instância e abasteça esse
db com os dados dos arquivo csv.
'''

In [17]:
# Configuração da conexão com o MySQL
config = {
    'user': 'root',
    'password': '',
    'host': '34.57.236.12',
    'database': 'mercado'
}

try:
    # Conexão com o banco de dados
    conn = mysql.connector.connect(**config)
    if conn.is_connected():
        print("Conexão bem-sucedida ao banco de dados!")

        # Criar um cursor
        cursor = conn.cursor()

        # Criar a tabela (se necessário)
        criar_tabela_query = """
        CREATE TABLE IF NOT EXISTS webcam (
            produto VARCHAR(255),
            preco DECIMAL(11,2),
            link VARCHAR(255),
            condicao VARCHAR(255),
            categoria VARCHAR(255)
        );
        """
        cursor.execute(criar_tabela_query)

        # Inserir os dados do DataFrame no banco de dados
        for _, row in df.iterrows():
            insert_query = """
            INSERT INTO webcam (produto, preco, link, condicao, categoria)
            VALUES (%s, %s, %s, %s, %s);
            """
            cursor.execute(insert_query, tuple(row))

        # Confirmar as mudanças no banco de dados
        conn.commit()
        print("Dados enviados com sucesso!")

except Error as e:
    print("Erro ao conectar ao banco de dados:", e)
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("Conexão encerrada.")

Conexão bem-sucedida ao banco de dados!
Dados enviados com sucesso!
Conexão encerrada.
